In [20]:
import boto3
import pathlib
from datetime import datetime
from typing import List, Tuple, Union, Dict

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import Column

from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Window
# from pyarrow.filesystem import S3FS


In [21]:
# master configuration to use only 4 CPU cores
spark = SparkSession.builder.master("local[4]").getOrCreate()

# basic configuration to use only a reasonable number of partitions
spark.conf.set("spark.sql.shuffle.partition", 4)

# configuration to work in UTC
spark.conf.set("spark.sql.session.timeZone", "UTC")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/12 12:15:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
env = "prod"
bucket = f"heroes-dl-bronze-{env}" 
prefix = "amazon/financial_events/financial_refund_events/part-00000-a06646ad-501c-4235-8b67-cf98b16213bb-c000.snappy.parquet"
filename = "downloads/downloaded_local_parquet.parquet" 

s3 = boto3.Session(profile_name=f"data-{env}").client("s3")

In [15]:
def download_parquet_from_s3(bucket, key, local_path):
    s3.download_file(bucket, key, local_path)


def print_parquet_schema(table):
    return table.schema

In [ ]:
download_parquet_from_s3(bucket, prefix, filename)

In [22]:
df = spark.read.parquet(filename)

In [28]:
df.printSchema()

root
 |-- region_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- marketplace_id: string (nullable = true)
 |-- posted_dt: date (nullable = true)
 |-- report_id: string (nullable = true)
 |-- raw_data: struct (nullable = true)
 |    |-- AmazonOrderId: string (nullable = true)
 |    |-- MarketplaceName: string (nullable = true)
 |    |-- PostedDate: string (nullable = true)
 |    |-- SellerOrderId: string (nullable = true)
 |    |-- ShipmentItemAdjustmentList: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- CostOfPointsReturned: struct (nullable = true)
 |    |    |    |    |-- CurrencyAmount: long (nullable = true)
 |    |    |    |    |-- CurrencyCode: string (nullable = true)
 |    |    |    |-- ItemChargeAdjustmentList: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- ChargeAmount: struct (nullable = true)
 |    |    |    |    |    |    |-- 

In [29]:
schema = df.printSchema
print(schema)

<bound method DataFrame.printSchema of DataFrame[region_id: string, account_id: string, marketplace_id: string, posted_dt: date, report_id: string, raw_data: struct<AmazonOrderId:string,MarketplaceName:string,PostedDate:string,SellerOrderId:string,ShipmentItemAdjustmentList:array<struct<CostOfPointsReturned:struct<CurrencyAmount:bigint,CurrencyCode:string>,ItemChargeAdjustmentList:array<struct<ChargeAmount:struct<CurrencyAmount:double,CurrencyCode:string>,ChargeType:string>>,ItemFeeAdjustmentList:array<struct<FeeAmount:struct<CurrencyAmount:double,CurrencyCode:string>,FeeType:string>>,ItemTaxWithheldList:array<struct<TaxCollectionModel:string,TaxesWithheld:array<struct<ChargeAmount:struct<CurrencyAmount:double,CurrencyCode:string>,ChargeType:string>>>>,OrderAdjustmentItemId:string,PromotionAdjustmentList:array<struct<PromotionAmount:struct<CurrencyAmount:double,CurrencyCode:string>,PromotionId:string,PromotionType:string>>,QuantityShipped:bigint,SellerSKU:string>>>, filename_adjusted: 

24/01/12 18:58:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 902911 ms exceeds timeout 120000 ms
24/01/12 18:58:35 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
# Print Parquet schema
# schema = print_parquet_schema(parquet_table)
# print(schema)